In [1]:
#the packages used
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [2]:
#checking the tables in the sqlite_db

db = sqlite3.connect('../data/ehr.sqlite')
db.execute("""SELECT 
    name
FROM 
    sqlite_schema
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';""").fetchall()

[('patients',), ('admission',), ('diagicd',), ('ictarget',)]

In [3]:
#ictarget

query = """
SELECT *
FROM ictarget
LIMIT 3

"""

with sqlite3.connect('../data/ehr.sqlite') as db: 
    ictar = pd.read_sql(query, db)
ictar

,unnamed:_0,subject_id,hadm_id,icd_e11,icd_e78,icd_e87,icd_f32,icd_i16,icd_i50,icd_n17,icd_y92,icd_z85
0,0,10000032,22841357,0,0,1,0,0,0,0,0,0
1,1,10000032,25742920,0,0,1,0,0,0,0,0,0
2,2,10000032,29079034,0,0,1,0,0,0,0,0,0


In [4]:
#ictarget

query = """
SELECT *
FROM patients
LIMIT 3

"""

with sqlite3.connect('../data/ehr.sqlite') as db: 
    patient = pd.read_sql(query, db)
patient

,unnamed:_0,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,min_year_group,max_year_group,min_age,base_anchor_year
0,0,10000032,F,52,2180,2014 - 2016,2180-09-09 00:00:00,2014,2016,44,2174
1,1,10000048,F,23,2126,2008 - 2010,0,2008,2010,21,2126
2,2,10000068,F,19,2160,2008 - 2010,0,2008,2010,17,2160


In [5]:
query="""
SELECT COUNT(DISTINCT subject_id) as patient_count
FROM ictarget
"""
with sqlite3.connect('../data/ehr.sqlite') as db: 
    pat_count = pd.read_sql(query, db)
pat_count

,patient_count
0,115012
